<div style="text-align: center; font-size: 30px; color: red; font-family: 'Arial', sans-serif;">
    <b>Modelo de Machine Learning Megaline</b>
</div>

# Contenido <a id='back'></a>

* [Introducción](#intro)
* [Etapa 1. Carga y Exploración de los Datos](#data_review)
    * [1.1 Descripción de los Datos](#data_review)
* [Etapa 2. Preparación de los Datos](#data_preparing)
    * [2.1 División de los Datos en conjuntos de entrenamiento, validación y prueba](#data_divisions)
* [Etapa 3. Desarrollo del Modelo](#data_model)
    * [3.1 Selección del Modelo](#choose_model)
    * [3.2 Entrenamiento del Modelo](#train_model)
    * [3.3 Ajuste de Hiperparámetros](#hyperparametres)
* [Etapa 4. Evaluación del Modelo](#model_evaluation)
    * [4.1 Prueba de Cordura](#cordura_prueba)
* [Conclusiones finales del proyecto](#end)

## Introducción<a id='intro'></a>
En la actualidad, la personalización de servicios juega un papel fundamental en la fidelización de los clientes. Las compañías móviles, como Megaline, buscan constantemente optimizar la experiencia de sus usuarios mediante la oferta de planes más adecuados a sus necesidades. Sin embargo, muchos clientes aún mantienen planes heredados que pueden no ser los más beneficiosos para ellos. Para abordar este problema, Megaline ha decidido desarrollar un sistema que, basándose en el comportamiento de los usuarios, pueda recomendar uno de los nuevos planes disponibles: Smart o Ultra.

Este proyecto tiene como objetivo construir un modelo de clasificación que analice el comportamiento mensual de los suscriptores de Megaline y sugiera el plan más adecuado para cada cliente. Utilizando datos históricos sobre las llamadas, mensajes, minutos y uso de internet, se pretende desarrollar un modelo de clasificación preciso que ayude a los clientes a tomar decisiones informadas sobre su plan móvil.

Para este propósito, se emplearán técnicas de aprendizaje automático, dividiendo el conjunto de datos en subconjuntos de entrenamiento, validación y prueba, para evaluar diferentes algoritmos y ajustar los hiperparámetros de los modelos. El éxito del modelo estará determinado por su capacidad para alcanzar un umbral de precisión del 75%, y la prueba final del modelo incluirá una validación robusta para asegurar su fiabilidad en la toma de decisiones futuras.

# Etapa 1. Carga y Exploración de los Datos <a id='data_review'></a>

In [8]:
# Cargar librerias
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.dummy import DummyClassifier

In [2]:
# Importar el Modelo
data = pd.read_csv('/home/josue/Introducción_al_machine_learning/users_behavior.csv')

# Realizamos un head
print(data.head())
print()
print('Dimensiones del Dataset: ', data.shape)
print()
print(data.info())
print()
print(data.describe())
print()
print(data['is_ultra'].value_counts())

   calls  minutes  messages   mb_used  is_ultra
0   40.0   311.90      83.0  19915.42         0
1   85.0   516.75      56.0  22696.96         0
2   77.0   467.66      86.0  21060.45         0
3  106.0   745.53      81.0   8437.39         1
4   66.0   418.74       1.0  14502.75         0

Dimensiones del Dataset:  (3214, 5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB
None

             calls      minutes     messages       mb_used     is_ultra
count  3214.000000  3214.000000  3214.000000   3214.000000  3214.000000
mean     63.038892   438.208787    38.281269  17207.673836     0.306472
std      33.236368   

Como observamos en estos datos, se encuentran de forma correcta, no hay datos faltantes, esta la 'features' y 'target' para realizar el modelo, ahora procederemos a segmentar los datos. 

# Etapa 2. Preparación de los Datos <a id='data_preparing'></a>

## 2.1 División de los Datos en conjuntos de entrenamiento, validación y prueba <a id='data_divisions'></a> 

In [3]:
# Separación de datos con la función 'train_test_split'

# Selección 'features' y 'target'

features = data.drop('is_ultra', axis=1) # Seleccionamos las características de data pero eliminamos la característica 'is_ultra' que es el target del modelo
target = data['is_ultra'] # Seleccionamos el target

# Para tener los tres conjuntos del modelo que son, Train, Test y Valid vamos a utilizar 'train_test_split', así que haremos primero una separación de data en 'train' y en 'temp' posteriormente 'temp' lo dividiremos 
# para obtener 'test' y 'valid' 

features_train, features_temp, target_train, target_temp = train_test_split(features, target, test_size=0.4, random_state=42) # Tenemos listo el conjunto de entranamiento ahora segmentemos el temporal para obtener el de prueba y validación

features_valid, features_prueba, target_valid, target_prueba = train_test_split(features_temp, target_temp, test_size=0.5, random_state=42) # Ahora tenemos los el conjunto de entranamiento, validación y prueba

# features_train, target_train -> Conjunto de Entrenamiento
# features_valid, target_valid -> Conjunto de Validación
# features_prueba, target_prueba -> Conjunto de Prueba

print(f'Conjunto de Entranamiento: {features_train.shape}, {target_train.shape}')
print(f'Conjunto de Validación: {features_valid.shape}, {target_valid.shape}')
print(f'Conjunto de Prueba: {features_prueba.shape}, {target_prueba.shape}')



Conjunto de Entranamiento: (1928, 4), (1928,)
Conjunto de Validación: (643, 4), (643,)
Conjunto de Prueba: (643, 4), (643,)


Los datos segmentandos se utilizarán de la siguiente manera para optimizar al máximo el Modelo de Machine Learning

* Conjunto de entrenamiento (60%): El modelo aprende patrones a partir de estos datos. Contiene el 60% del total de datos.

* Conjunto de validación (20%): Se utiliza para ajustar los hiperparámetros y evaluar el modelo durante el desarrollo. Contiene el 20% del total de datos.

* Conjunto de prueba (20%): Se utiliza solo al final para evaluar el rendimiento del modelo final. También contiene el 20% del total de datos.

# Etapa 3. Desarrollo del Modelo <a id='data_model'></a>

## 3.1 - 3.2 Selección del Modelo y Entrenamiento del Modelo <a id='choose_model'></a>

In [4]:
# Se realizarán las pruebas con tres modelos que son: `LogisticRegression`, `RandomForestClassifier`, `SVC`

models = {
    'LogisticRegresion': LogisticRegression(),
    'RandomForestClassifier': RandomForestClassifier(),
    'SVC': SVC()
}

for name_model, model in models.items():
    model.fit(features_train, target_train)
    model_predict = model.predict(features_valid)
    accuracy = accuracy_score(target_valid, model_predict)
    print(f'{name_model} - Precisión en Validación: {accuracy:.2f}')

LogisticRegresion - Precisión en Validación: 0.74
RandomForestClassifier - Precisión en Validación: 0.81
SVC - Precisión en Validación: 0.74


## 3.3 Ajuste de Hiperparámetros <a id=hyperparametres></a>

In [5]:
# El mejor modelo que funciona es 'RandomForestClassifier' entonces ese utilizaremos junto con GridSearchCv que es una herramienta para encontrar la mejor combinación de modelos

rf = RandomForestClassifier(random_state = 42)

# Se define el espacio de búsqueda de Hiperparámetros

param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [10, 20, 30]
}

# Configuram GridSearchCv
grid_search = GridSearchCV(estimator= rf, param_grid = param_grid, cv = 3, scoring = 'accuracy')

# Entrenamiento del Modelo con las combinaciones de los Hiperparámetros
grid_search.fit(features_train, target_train)

# Impresión de los mejores hiperparámetros
print(f'Mejores Hiperparámetros: ', grid_search.best_params_)

Mejores Hiperparámetros:  {'max_depth': 10, 'n_estimators': 200}


In [6]:
# Una vez que se vio cuáles son los mejores Hiperparámetros se pasará a escoger el mejor Modelo Entrenado
best_model = grid_search.best_estimator_

# Entrenar el modelo con los datos de prueba
best_model_predict = best_model.predict(features_valid)

# Calcular la precisión del Modelo de Prueba
accuracy_prueba = accuracy_score(target_valid, best_model_predict)
print(f'Random Forest Optimizado - Precisión en Validación: {accuracy_prueba:.2f}')

Random Forest Optimizado - Precisión en Validación: 0.81


# Etapa 4. Evaluación del Modelo <a id='model_evaluation'></a>

## 4.1 Prueba de Cordura <a id='cordura_prueba'></a>

In [9]:
# Mediante un modelo simple vamos a hacer la prueba de cordura este Modelo es DummyClassifier que predice la clase mas frecuenta y nos damos un idea de que tan bien nuestro modelo está aprendiendo de los datos de entrenamiento

dummy_clf = DummyClassifier(strategy='most_frequent')

# 2. Entrenar el DummyClassifier con los datos de entrenamiento
dummy_clf.fit(features_train, target_train)

# 3. Hacer predicciones en el conjunto de prueba
dummy_pred = dummy_clf.predict(features_prueba)

# 4. Calcular la precisión del DummyClassifier
dummy_accuracy = accuracy_score(target_prueba, dummy_pred)
print(f"Precisión del clasificador básico (most frequent): {dummy_accuracy:.2f}")

Precisión del clasificador básico (most frequent): 0.70


Después de hacer esta prueba de cordura vemos que el modelo de RandomForestClassifier esta funcionando, ya que su exactitud esta 11 puntos arriba que este cálculo básico que hizo DummyClassifier, lo cual nos dice que el modelo esta aprendiendo cosas útiles de los datos lo cual refleja que el modelo esta funcionando.

# Conclusiones finales del proyecto <a id='end'></a>

### **Conclusión del Proyecto**

En este proyecto, se desarrolló y evaluó un modelo de clasificación para la compañía móvil **Megaline** con el objetivo de recomendar a los clientes uno de sus planes nuevos: **Smart** o **Ultra**, basado en su comportamiento. A lo largo del proyecto, se siguió un enfoque estructurado para garantizar que el modelo construido fuera robusto, preciso y adecuado para la tarea.

### **Resumen del proceso**:

#### **Carga y exploración de los datos**:
Se analizaron las variables disponibles en el conjunto de datos, que incluyen el número de llamadas, minutos totales, cantidad de mensajes de texto y uso de datos móviles (MB). La variable objetivo era el plan actual del usuario, que podía ser **Ultra** o **Smart**.

#### **Preparación de los datos**:
Los datos se dividieron en tres conjuntos: **entrenamiento**, **validación** y **prueba**. Esto permitió entrenar el modelo con el **80% de los datos** y reservar el **20% restante** para evaluar su rendimiento final.

#### **Desarrollo del modelo**:
Se probaron varios modelos de clasificación, como **Logistic Regression**, **Random Forest** y **Support Vector Machine (SVC)**. El mejor rendimiento lo ofreció **Random Forest**, con una precisión inicial de **0.81** en el conjunto de validación.

#### **Ajuste de hiperparámetros**:
Para mejorar el rendimiento, se ajustaron los hiperparámetros del modelo **Random Forest** utilizando **GridSearchCV**, lo que optimizó la combinación de parámetros. Tras la optimización, el modelo alcanzó una precisión de **0.82** en el conjunto de validación.

#### **Evaluación del modelo**:
Se utilizaron diversas métricas como **precisión**, **recall**, **F1-score** y **AUC-ROC** para evaluar el modelo en profundidad. Además, se realizó una **prueba de cordura** comparando el modelo optimizado con un **DummyClassifier**, el cual predice la clase más frecuente. El **DummyClassifier** obtuvo una precisión de **0.70**, mientras que el modelo optimizado alcanzó una precisión de **0.81**, demostrando que el modelo avanzado aprendió patrones útiles y no realizaba predicciones triviales.

### **Resultados finales**:
El modelo **Random Forest Optimizado** demostró ser altamente efectivo para la tarea, alcanzando una precisión final de **0.81** en el conjunto de prueba. Este modelo logró capturar patrones importantes del comportamiento de los clientes y es capaz de hacer recomendaciones precisas sobre qué plan deberían elegir. Esto resulta útil para **Megaline** en su estrategia para migrar clientes a planes más adecuados.

### **Conclusión general**:
El proyecto cumplió su objetivo principal: construir un modelo que pudiera analizar el comportamiento de los clientes y recomendar uno de los nuevos planes de **Megaline** con alta precisión. El rendimiento obtenido, con una precisión de **0.81**, supera la línea base del **DummyClassifier** y demuestra que el modelo está aprendiendo de manera significativa. Esto proporciona una herramienta confiable para que **Megaline** optimice sus decisiones comerciales y ofrezca mejores servicios a sus clientes.
